# **Searching Descriptions**

# Setup
The cells below will help to set up the rest of the notebook. 

I'll start by configuring the kernel that's running this notebook:

In [1]:
# Change the cwd
%cd ..

# Enable the autoreload module
%load_ext autoreload
%autoreload 2

/Users/thubbard/Documents/Personal/Programming/pax-east-game-search


/Users/thubbard/Documents/Personal/Programming/pax-east-game-search/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Next, I'm going to import the necessary modules:

In [8]:
# General import statements
import pandas as pd

# Set up the OpenAI API client
from openai import OpenAI
client = OpenAI()

# Loading Data

In [7]:
# Load the data
pax_east_data_df = pd.read_excel("data/pax-east-2024-games.xlsx")

# Show a random sample
pax_east_data_df.sample(5)

,id,publisher,booth,ganme_name,description,link
180,181,Stanion Studios,11031,TideTurn,"TideTurn, produced by Stanion Studios, is a dy...",https://www.playtideturn.com/index.html
119,120,High Point University,10049,No Info,"""Students from the Game Design major, the Game...",https://www.highpoint.edu/gamedesign/
104,105,Galaktus Publishing,14049,Union Of Gnomes \n(HeroCraft),"Lead the unbreakable gnomes union, uprise agai...",https://store.steampowered.com/app/2097030/Uni...
207,208,Cephalofair Games,12115,No Info,No Info,https://cephalofair.com/pages/shop
162,163,Schell Games,13093,Silent Slayer: Vault of the Vampire,In this single-player horror game filled with ...,https://www.meta.com/experiences/5353670608005...


# Embedding Descriptions

In [11]:
def embed_text(text):
    """
    Embeds some text using the OpenAI text-embedding-3 model.
    """
    response = client.embeddings.create(input=text, model="text-embedding-3-large")
    return response.data[0].embedding

In [13]:
# Copy the DataFrame to operate on
pax_east_data_with_emb_df = pax_east_data_df.copy()

# Embed the game descriptions
pax_east_data_with_emb_df["description_emb"] = pax_east_data_with_emb_df["description"].apply(embed_text)

### **SAVING**

In [39]:
# Save the pax_east_data_with_emb_df to json
pax_east_data_with_emb_df.to_json("data/pax-east-2024-games-emb.json")

In [61]:
pax_east_data_with_emb_df["booth"] = pax_east_data_with_emb_df["booth"].apply(lambda x: str(x))

In [62]:
# Save the data as a .parquet file
pax_east_data_with_emb_df.to_parquet("data/pax-east-2024-games-emb.parquet")

# Searching Descriptions
Below, you'll be able to search for a game. Type in your query, and then find the games that are similar. 

In [55]:
# Declare your query, and embed it 
query = """bloons tower defense"""
query_emb = embed_text(query)

In [56]:
# Copy the DataFrame to operate on
sim_df = pax_east_data_with_emb_df.copy()

# Calculate the similarity between the query and the game descriptions
import numpy as np
sim_df["similarity"] = sim_df["description_emb"].apply(lambda x: np.dot(query_emb, x))


In [57]:
# Sort the DataFrame by similarity
sim_df = sim_df.sort_values("similarity", ascending=False)

# For each of the first 5 most similar games, print the title and the similarity
for i in range(5):
    print(f"Title: {sim_df.iloc[i]['ganme_name']}")
    print(f"Similarity: {sim_df.iloc[i]['similarity']}")
    print("Description:")
    print(sim_df.iloc[i]['description'])
    print("\n")

Title: Everwarder
Similarity: 0.4170814875888395
Description:
A 2D roguelite tower defense. Defend the crystal against endless 
streams of enemies while exploring procedurally generated worlds. 
Design your defense and keep up with constantly rising difficulty 
by acquiring powerful artifacts, evolve units, progress in upgrade 
tree and more!


Title: Cats on Duty 
(ESDigital Games)
Similarity: 0.37629605598971494
Description:
A purr-fect blend of RTS, tower defense, and match-3 game 
genres. Manage two game fields at once as well as arm and 
upgrade your cat towers to defeat hordes of formidable foes. 
Enjoy the meow-nificent battle!


Title: Zombie Rollerz: The Last Ship
Similarity: 0.37146815102555497
Description:
Build turrets, lead your crew, and upgrade the ship while gathering 
loot in this exciting "Tower Defense-on-wheels zombie survival 
roguelite" game.


Title: Back Off Barbarian
Similarity: 0.3372287254781051
Description:
See: Castle Crashers


Title: Toy Tactics
Similarit